In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

***Importing Required Packages***

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

***Reading The Datasets***

In [ ]:
train=pd.read_csv("../input/competitive-data-science-predict-future-sales/sales_train.csv")
items=pd.read_csv("../input/competitive-data-science-predict-future-sales/items.csv")
categories=pd.read_csv("../input/competitive-data-science-predict-future-sales/item_categories.csv")
shops=pd.read_csv("../input/competitive-data-science-predict-future-sales/shops.csv")
test=pd.read_csv("../input/competitive-data-science-predict-future-sales/test.csv")

**Understanding The Datasets**

In [ ]:
train.shape

In [ ]:
train.head()

In [ ]:
train.info()

In [ ]:
items.shape

In [ ]:
items.head()

In [ ]:
categories.shape

In [ ]:
categories.head()

In [ ]:
shops.shape

In [ ]:
shops.head()

In [ ]:
train.date_block_num.unique()

***Dropping "date column from train ***

In [ ]:
train.drop("date",axis=1,inplace=True)

**Correlation**

In [ ]:
train.corr()

***Visualization Of the data***

In [ ]:
sns.heatmap(train.corr())

**Defining Hist plot such that we can use in future**

In [ ]:
def hist(data, x, bins, title, xlabel):
    plt.figure(figsize = (15, 12))
    sns.set()
    sns.distplot(data[x], color = 'blue')
    plt.title(title)
    plt.xlabel(xlabel)
    plt.show()

***Visualizing the data***

In [ ]:
hist(train,"date_block_num",1,"date_block_num_dp","date_block_num")

In [ ]:
hist(train,"shop_id",1,"shop_id_dp","shop_id")

In [ ]:
hist(train,"item_id",1,"item_id_dp","item_id")

In [ ]:
hist(train,"item_price",1000,"item_price_dp","item_price")

In [ ]:
plt.figure(figsize = (12, 8))
sns.boxplot(y=train['item_price'])
plt.ylim(0, 10000)
plt.grid()
plt.title('Boxplot for Item_price')
plt.ylabel('item_price')

In [ ]:
test.head()

**Dropping unneeded column**

In [ ]:
train.drop("date_block_num",axis=1,inplace=True)

**Taking x,y**

In [ ]:
x_train=train[["shop_id","item_id"]]
y_train=train["item_cnt_day"]

In [ ]:
x_test=test[["shop_id","item_id"]]

**Splitting the data set into train set and validation set so that we can know how our model will work in the test dattaset**

In [ ]:
from sklearn.model_selection import train_test_split
x_train1,x_val,y_train1,y_val = train_test_split(x_train,y_train,test_size = .2,random_state =42)

***Linear Regression***

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import confusion_matrix

In [ ]:
modlr=LinearRegression()
modlr=modlr.fit(x_train,y_train)

In [ ]:
rsq=modlr.score(x_train,y_train)
rsq

In [ ]:
rsq=modlr.score(x_val,y_val)
rsq

Its doing very bad

***CatBoost and Light GBM***

In [ ]:
from catboost import CatBoostRegressor
import lightgbm as lgb

In [ ]:
lgb_train = lgb.Dataset(x_train, y_train)   
lgb_eval = lgb.Dataset(x_val, y_val, reference=lgb_train)

In [ ]:
params = {'boosting_type': 'gbdt',
              'objective': 'regression',
              'num_leaves': 40,
              'learning_rate': 0.1,
              'feature_fraction': 0.9
              }
gbm = lgb.train(params,
    lgb_train,
    num_boost_round=200,
    valid_sets=[lgb_train, lgb_eval],
    valid_names=['train','valid'])

In [ ]:
y_te=gbm.predict(x_test).clip(0,40)
y_te

In [ ]:
submission = pd.DataFrame({
    'ID': test.index, 
    'item_cnt_month': y_te
})
submission.to_csv('sales_submission.csv', index=False)
print(submission)